In [1]:
import json
import os
from datetime import datetime
#from rtf_utils.moat_utils import MoatTile
from rtf_utils.dfa_utils import CampaignManagerReport,get_dfa_report,clean_dcm_file
from rtf_utils.gcp_utils import BigQuery,CloudStorage

from essence.analytics.platform import securedcredentials as secure_creds
import time

import warnings
warnings.filterwarnings("ignore", "Your application has authenticated using end user credentials")

In [ ]:
with open('dags/brand_report_task_queue.json') as json_file:
    config_json = json.load(json_file)
    campaigns = config_json['campaigns']

Task = (report_id, bq_dest (proj:dataset.table), schema)

In [2]:
start_date='2019-09-01'
end_date='2019-09-10'
report_id = 631165400

In [3]:
def dfa_report_extract(report_id,start_date,end_date,**context):
    ## staging bucket = context['staging_bucket'] in airflow
    
    bucket_name = 'rtf_staging'
    
    PROJECT_ID='essence-analytics-dwh'
    service_account_email='131786951123-compute@developer.gserviceaccount.com'
    credentialsFromVault=secure_creds.getCredentialsFromEssenceVault(service_account_email)
    
    local_filename = get_dfa_report(credentialsFromVault,
                                report_id,#631165400
                                start_date,
                                end_date)
    print("Cleaning File")
    clean_dcm_file(local_filename)
    
    print("Auth GCS")
    gcs = CloudStorage(credentialsFromVault)
    folder = "brand_reporting/"
    destination_blob_name = folder + local_filename
          
    
    print("Upload File")
    
    gcs.upload_blob(bucket_name, destination_blob_name, local_filename, mode='filename')
    
    stored_blob = gcs.get_blob(bucket_name,destination_blob_name)
          
    print("Clean Up Local")
    os.remove(local_filename)
    
    return "gs://" + stored_blob.bucket.name + "/" + stored_blob.name

In [7]:
#def dfa_report_load(pull_task_id,dataset_table,schema=None,**context)

def dfa_report_load(file_uri,dataset_table,schema=None,**context):
    ##context['ti'].xcom_pull('pull_task_id')
    
    print("Get Creds from Vault")
    service_account_email='131786951123-compute@developer.gserviceaccount.com'
    credentialsFromVault=secure_creds.getCredentialsFromEssenceVault(service_account_email)
    
    
    
    dataset_id = dataset_table.split(".")[0]
    dest_table = dataset_table.split(".")[1]
    
    print("Auth BQ")
    bq = BigQuery(credentialsFromVault)
    
    print("Load to BQ")
    job = bq.load_from_gcs(dataset_id,file_uri,dest_table,mode='Append')

In [7]:
resp = dfa_report_extract(report_id,start_date,end_date)

Get Report Meta
File is already available. Start download
File downloaded to Bonito_Classification.csv
Cleaning File
Auth GCS
Upload File
Clean Up Local


In [6]:
dfa_report_load("gs://rtf_staging/brand_reporting/Bonito_Classification.csv","RTF_DWH_CampaignManager.STAGING_classification_report")

Get Creds from Vault
Auth BQ
Load to BQ
